In [ ]:
pip install selenium

In [2]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [51]:
pip install pandas openpyxl selenium webdriver-manager

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
     ---- -------------------------------- 30.7/250.0 kB 262.6 kB/s eta 0:00:01
     ---------- -------------------------- 71.7/250.0 kB 491.5 kB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install gc

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement gc (from versions: none)
ERROR: No matching distribution found for gc


In [2]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import gc

options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
flag = 0
driver.get("https://www.cbioportal.org/")

time.sleep(5)

ele = driver.find_elements("xpath", "/html/body/div[1]/div/div[3]/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div[1]/div/div/div/div[2]/div[1]/ul/li[33]")
ele[0].click()

time.sleep(1)
ul = driver.find_elements("xpath", "/html/body/div[1]/div/div[3]/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div[1]/div/div/div/div[2]/div[2]/ul/ul/ul[1]/li[9]/label/span")
ul[0].click()
time.sleep(4)

exportData = driver.find_elements("xpath", "/html/body/div[1]/div/div[3]/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div[2]/a[2]")
exportData[0].click()
time.sleep(4)

clinicalData = driver.find_elements("xpath", "/html/body/div/div/div[3]/div/div/div/div/div[1]/div/div[2]/div[1]/ul/li[2]/a")
clinicalData[0].click()
time.sleep(4)
wait = WebDriverWait(driver, 10) 

while True:
    # showMore = driver.find_elements("xpath","/html/body/div[1]/div/div[3]/div/div/div/div/div[1]/div/div[2]/div[1]/div/div[2]/span/div/div/div[3]/div/div/button[2]")
    showMore = wait.until(EC.presence_of_element_located(("id","showMoreButton")))
    is_disabled = showMore.get_attribute('disabled')

    # If 'disabled' is present and not None, or the button explicitly has 'disabled' set to true, break the loop
    if is_disabled is not None and is_disabled != 'false':
        print("Button is disabled, stopping the loop.")
        break
    
    # Click the "Show More" button if not disabled
    showMore.click()
    time.sleep(1)


table = driver.find_element("xpath", "/html/body/div[1]/div/div[3]/div/div/div/div/div[1]/div/div[2]/div[1]/div/div[2]/span/div/div/div[2]/table")  # Replace with your table's XPath
print("Table initialized")
# Initialize a list to store all rows of data
table_data = []

columns = ['Patient ID', 'Sample ID', 'Cancer Type Detailed', 'Overall Survival Status', 'Disease Free Status', 
           'Disease-specific Survival status', 'Progression Status', 'Mutation Count', 'Fraction Genome Altered', 
           'Diagnosis Age', 'MSI MANTIS Score', 'MSIsensor Score', 'Ethnicity Category', 
           'Race Category', 'Subtype', 'Tumor Type', 'ICD-O-3 Histology Code', 'ICD-O-3 Site Code', 
           'AJCC Publication Version Type', 'Birth from Initial Pathologic Diagnosis Date', 'Buffa Hypoxia Score',
           'Disease Free (Months)', 'Form completion date', 'Genetic Ancestry Label', 'ICD-10 Classification',
           'In PanCan Pathway Analysis', 'Informed consent verified', 'Last Alive Less Initial Pathologic Diagnosis Date Calculated Day Value',
           'Last Communication Contact from Initial Pathologic Diagnosis Date', 'Months of disease-specific survival',
           'Neoadjuvant Therapy Type Administered Prior To Resection Text', 'New Neoplasm Event Post Initial Therapy Indicator',
           'Number of Samples Per Patient', 'Other Patient ID', 'Overall Survival (Months)', 'Patient Weight', 'Person Neoplasm Cancer Status',
           'Prior Diagnosis', 'Progress Free Survival (Months)', 'Radiation Therapy', 'Ragnum Hypoxia Score', 'Sex', 'TCGA PanCanAtlas Cancer Type Acronym',
           'Winter Hypoxia Score', 'Sample Type', 'Aneuploidy Score', 'Neoplasm Histologic Grade', 'Oncotree Code',
           'Somatic Status', 'Tissue Prospective Collection Indicator', 'Tissue Retrospective Collection Indicator',
           'Tissue Source Site', 'Tissue Source Site Code', 'TMB (nonsynonymous)', 'Tumor Disease Anatomic Site']

# Initialize an empty DataFrame with these columns
df = pd.DataFrame(columns=columns)
k = 0
rows = table.find_elements(By.TAG_NAME, "tr")

while k < len(rows):

    try:
        row = rows[k]
        cells = row.find_elements(By.TAG_NAME, "td")
    
        row_data = [cell.text for cell in cells]
        
        if row_data:

            sample_dict = {
                'Patient ID': row_data[0], 'Sample ID': row_data[1], 'Cancer Type Detailed': row_data[2], 'Overall Survival Status': row_data[3], 'Disease Free Status': row_data[4],
               'Disease-specific Survival status': row_data[5], 'Progression Status': row_data[6], 'Mutation Count': row_data[7], 'Fraction Genome Altered': row_data[8],
               'Diagnosis Age': row_data[9], 'MSI MANTIS Score': row_data[10], 'MSIsensor Score': row_data[11], 'Overall Survival Status': row_data[12], 'Ethnicity Category': row_data[13],
               'Race Category': row_data[14], 'Subtype': row_data[15], 'Tumor Type': row_data[16], '"International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code"': row_data[17],
               '"International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code"': row_data[18],'American Joint Committee on Cancer Publication Version Type': row_data[19],
               'Birth from Initial Pathologic Diagnosis Date': None, 'Buffa Hypoxia Score': None,
               'Disease Free (Months)': None, 'Form completion date': None, 'Genetic Ancestry Label': None, 'ICD-10 Classification': None,
               'In PanCan Pathway Analysis': None, 'Informed consent verified': None, 'Last Alive Less Initial Pathologic Diagnosis Date Calculated Day Value': None,
               'Last Communication Contact from Initial Pathologic Diagnosis Date': None, 'Months of disease-specific survival': None,
               'Neoadjuvant Therapy Type Administered Prior To Resection Text': None, 'New Neoplasm Event Post Initial Therapy Indicator': None,
               'Number of Samples Per Patient': None, 'Other Patient ID': None, 'Overall Survival (Months)': None, 'Patient Weight': None, 'Person Neoplasm Cancer Status': None,
               'Prior Diagnosis': None, 'Progress Free Survival (Months)': None, 'Radiation Therapy': None, 'Ragnum Hypoxia Score': None, 'Sex': None, 'TCGA PanCanAtlas Cancer Type Acronym': None,
               'Winter Hypoxia Score': None, 'Sample Type': None, 'Aneuploidy Score': None, 'Neoplasm Histologic Grade': None, 'Oncotree Code': None,
               'Somatic Status': None, 'Tissue Prospective Collection Indicator': None, 'Tissue Retrospective Collection Indicator': None,
               'Tissue Source Site': None, 'Tissue Source Site Code': None, 'TMB (nonsynonymous)': None, 'Tumor Disease Anatomic Site': None
            }
        
        links = row.find_elements(By.TAG_NAME, "a")   
   
        if len(links) > 1:  
            link = links[1].get_attribute("href")

            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[1])
            driver.get(link)
            time.sleep(5)
            clinData = driver.find_element("xpath", "/html/body/div/div/div[3]/div/div/div/div/div/div[2]/ul/li[3]/a")
            clinData.click()
            time.sleep(4)
        
            #Need to write code to get data from table
            sample_patient_table = driver.find_element("xpath", "/html/body/div[1]/div/div[3]/div/div/div/div/div/div[2]/div/div/div[1]/div/div[2]/table")
            sample_patient_rows = sample_patient_table.find_elements(By.TAG_NAME, "tr")
        
            for sample_patient_row in sample_patient_rows:
                cells = sample_patient_row.find_elements(By.TAG_NAME, "td")
                if cells:
                    temp_col_name = cells[0].text
                    temp_data = cells[1].text
                    sample_dict[temp_col_name] = temp_data

            sample_sample_table = driver.find_element("xpath", "/html/body/div/div/div[3]/div/div/div/div/div/div[2]/div/div/div[2]/div/div[2]/table")
            sample_sample_rows = sample_sample_table.find_elements(By.TAG_NAME, "tr")
        
            for sample_sample_row in sample_sample_rows:
                cells = sample_sample_row.find_elements(By.TAG_NAME, "td")
                if cells:
                    temp_col_name = cells[0].text
                    temp_data = cells[1].text
                    sample_dict[temp_col_name] = temp_data
        
            df.loc[len(df)] = sample_dict

















        

        
            # Find the div element
            timeline_div = driver.find_element("xpath", "/html/body/div[1]/div/div[3]/div/div/div/div/div/div[2]/div/div/div[3]")

            # Get all h3 tags within the div
            h3_tags = timeline_div.find_elements(By.TAG_NAME, "h3")

            # Get all tables within the div
            tables = timeline_div.find_elements(By.TAG_NAME, "table")

            # Process h3 tags
            for i in range(len(h3_tags)):
                if h3_tags[i].text == 'Status':
                    status_dict = {'Sample ID': row_data[1]}
                    status_headers_list = ['Sample ID']
                    status_table_data = []

                    # status_table = driver.find_element("xpath", "/html/body/div[1]/div/div[3]/div/div/div/div/div/div[2]/div/div/div[3]/div[1]/div[2]/table")
                    status_table = tables[i]
                    # status_thead = status_table.find_element(By.TAG_NAME, "thead")
                    # status_header_row = status_thead.find_element(By.TAG_NAME, "tr")
                    status_headers = status_table.find_elements(By.TAG_NAME, "th")
                    status_rows = status_table.find_elements(By.TAG_NAME, "tr")
                    for status_header in status_headers:
                        status_header_cells = []
                        status_header_cells.append(status_header.find_element(By.TAG_NAME, "span").find_element(By.TAG_NAME, "span"))
                        for cell in status_header_cells:
                            if cell.text not in status_dict.keys():
                                status_dict[cell.text] = None
                                status_headers_list.append(cell.text)

                    status_dicts = []
                    for status_row in status_rows:
                        cells = status_row.find_elements(By.TAG_NAME, "td")
                        status_row_data = [row_data[1]]
                        # status_row_data = [cell.text for cell in cells]
                        for cell in cells:
                            status_row_data.append(cell.text)
                        if len(status_row_data)<2:
                            continue
                        for i in range(len(status_headers_list)):
                            status_dict[status_headers_list[i]] = status_row_data[i]
                        status_dicts.append(status_dict.copy())

                    status_df = pd.DataFrame(columns = status_headers_list)
                    for status_d in status_dicts: 
                        status_df.loc[len(status_df)] = status_d
                    status_excel_file = 'StatusTableUterineCancerData.xlsx'
                    try:
                        existing_status_df = pd.read_excel(status_excel_file)
                    except FileNotFoundError:
                        existing_status_df = pd.DataFrame(columns=status_headers_list)

                    # Append the new data to the existing DataFrame
   
                    updated_status_df = existing_status_df._append(status_df, ignore_index=True)

                    # Save the updated DataFrame back to the Excel file
                    updated_status_df.to_excel(status_excel_file, index=False)
                elif h3_tags[i].text == 'Sample Acquisition':
                    sa_dict = {'Sample ID': row_data[1]}
                    sa_headers_list = ['Sample ID']
                    sa_table_data = []

                    # sa_table = driver.find_element("xpath", "/html/body/div[1]/div/div[3]/div/div/div/div/div/div[2]/div/div/div[3]/div[1]/div[2]/table")
                    sa_table = tables[i]
                    # sa_thead = sa_table.find_element(By.TAG_NAME, "thead")
                    # sa_header_row = sa_thead.find_element(By.TAG_NAME, "tr")
                    sa_headers = sa_table.find_elements(By.TAG_NAME, "th")
                    sa_rows = sa_table.find_elements(By.TAG_NAME, "tr")
                    for sa_header in sa_headers:
                        sa_header_cells = []
                        sa_header_cells.append(sa_header.find_element(By.TAG_NAME, "span").find_element(By.TAG_NAME, "span"))
                        for cell in sa_header_cells:
                            if cell.text not in sa_dict.keys():
                                sa_dict[cell.text] = None
                                sa_headers_list.append(cell.text)

                    sa_dicts = []
                    for sa_row in sa_rows:
                        cells = sa_row.find_elements(By.TAG_NAME, "td")
                        sa_row_data = [row_data[1]]
                        # sa_row_data = [cell.text for cell in cells]
                        for cell in cells:
                            sa_row_data.append(cell.text)
                        if len(sa_row_data)<2:
                            continue
                        for i in range(len(sa_headers_list)):
                            sa_dict[sa_headers_list[i]] = sa_row_data[i]
                        sa_dicts.append(sa_dict.copy())

                    sa_df = pd.DataFrame(columns = sa_headers_list)
                    for sa_d in sa_dicts: 
                        sa_df.loc[len(sa_df)] = sa_d
                    sa_excel_file = 'SATableUterineCancerData.xlsx'
                    try:
                        existing_sa_df = pd.read_excel(sa_excel_file)
                    except FileNotFoundError:
                        existing_sa_df = pd.DataFrame(columns=sa_headers_list)

                    # Append the new data to the existing DataFrame
   
                    updated_sa_df = existing_sa_df._append(sa_df, ignore_index=True)

                    # Save the updated DataFrame back to the Excel file
                    updated_sa_df.to_excel(sa_excel_file, index=False)
                elif h3_tags[i].text == 'Treatment':
                    treatment_dict = {'Sample ID': row_data[1]}
                    treatment_headers_list = ['Sample ID']
                    treatment_table_data = []

                    # treatment_table = driver.find_element("xpath", "/html/body/div[1]/div/div[3]/div/div/div/div/div/div[2]/div/div/div[3]/div[1]/div[2]/table")
                    treatment_table = tables[i]
                    # treatment_thead = treatment_table.find_element(By.TAG_NAME, "thead")
                    # treatment_header_row = treatment_thead.find_element(By.TAG_NAME, "tr")
                    treatment_headers = treatment_table.find_elements(By.TAG_NAME, "th")
                    treatment_rows = treatment_table.find_elements(By.TAG_NAME, "tr")
                    for treatment_header in treatment_headers:
                        treatment_header_cells = []
                        treatment_header_cells.append(treatment_header.find_element(By.TAG_NAME, "span").find_element(By.TAG_NAME, "span"))
                        for cell in treatment_header_cells:
                            if cell.text not in treatment_dict.keys():
                                treatment_dict[cell.text] = None
                                treatment_headers_list.append(cell.text)

                    treatment_dicts = []
                    for treatment_row in treatment_rows:
                        cells = treatment_row.find_elements(By.TAG_NAME, "td")
                        treatment_row_data = [row_data[1]]
                        # treatment_row_data = [cell.text for cell in cells]
                        for cell in cells:
                            treatment_row_data.append(cell.text)
                        if len(treatment_row_data)<2:
                            continue
                        for i in range(len(treatment_headers_list)):
                            treatment_dict[treatment_headers_list[i]] = treatment_row_data[i]
                        treatment_dicts.append(treatment_dict.copy())

                    treatment_df = pd.DataFrame(columns = treatment_headers_list)
                    for treatment_d in treatment_dicts: 
                        treatment_df.loc[len(treatment_df)] = treatment_d
                    treatment_excel_file = 'TreatmentTableUterineCancerData.xlsx'
                    try:
                        existing_treatment_df = pd.read_excel(treatment_excel_file)
                    except FileNotFoundError:
                        existing_treatment_df = pd.DataFrame(columns=treatment_headers_list)

                    # Append the new data to the existing DataFrame
   
                    updated_treatment_df = existing_treatment_df._append(treatment_df, ignore_index=True)

                    # Save the updated DataFrame back to the Excel file
                    updated_treatment_df.to_excel(treatment_excel_file, index=False)  
                else:
                    print("No status, sample acquisition or treatment tables")


        #STATUS TABLE CODE ---------------------------------------------------------------------------------------------------------------------------

            
        #STATUS TABLE CODE END -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

            driver.close()
            main_window_handle = driver.window_handles[0]
            driver.switch_to.window(main_window_handle)
        print("row "+str(k)+" processed" )    
        k=k+1 
        gc.collect()
    except Exception as e:
        print(e)
        driver.close()
        main_window_handle = driver.window_handles[0]
        driver.switch_to.window(main_window_handle)
        
        print("Retrying in 2 minutes...")
        time.sleep(120) 
        driver.refresh()
        time.sleep(4)
print("Done with for")
time.sleep(4)

















# df = pd.DataFrame(table_data, columns=['Patient ID', 'Sample ID', 'Cancer Type Detailed', 'Overall Survival Status', 'Disease Free Status', 'Disease-specific Survival status', 'Progression Status', 'Mutation Count', 'Fraction Genome Altered', 'Diagnosis Age', 'MSI MANTIS Score', 'MSIsensor Score', 'Overall Survival Status', 'Ethnicity Category', 'Race Category', 'Subtype', 'Tumor Type', '"International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code"', '"International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code"','American Joint Committee on Cancer Publication Version Type'])

# Write the DataFrame to an Excel file
excel_filename = 'UterineCancerData.xlsx'
try:
    existing_df = pd.read_excel(excel_filename)
except FileNotFoundError:
    existing_df = pd.DataFrame(columns=columns)

# Append the new data to the existing DataFrame
updated_df = existing_df._append(df, ignore_index=True)

# Save the updated DataFrame back to the Excel file
updated_df.to_excel(excel_filename, index=False)
print('Data appended to existing file:', excel_filename)

Button is disabled, stopping the loop.
Table initialized
row 0 processed
row 1 processed
row 2 processed
row 3 processed
row 4 processed
row 5 processed
'NoneType' object has no attribute 'click'


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.71)
Stacktrace:
	GetHandleVerifier [0x00998C93+51395]
	(No symbol) [0x00905EF1]
	(No symbol) [0x007BE13A]
	(No symbol) [0x007A25F2]
	(No symbol) [0x00819B2B]
	(No symbol) [0x0081E9D4]
	(No symbol) [0x00813E53]
	(No symbol) [0x007EC629]
	(No symbol) [0x007ED40D]
	GetHandleVerifier [0x00D16453+3711107]
	GetHandleVerifier [0x00D5583A+3970154]
	GetHandleVerifier [0x00D50B28+3950424]
	GetHandleVerifier [0x00A49C99+776393]
	(No symbol) [0x009116C4]
	(No symbol) [0x0090C5A8]
	(No symbol) [0x0090C759]
	(No symbol) [0x008FDD80]
	BaseThreadInitThunk [0x75867BA9+25]
	RtlInitializeExceptionChain [0x76F9BD2B+107]
	RtlClearBits [0x76F9BCAF+191]
